In [ ]:
def get_rev_plot_arr(concat_sess):
    
    trialmat = concat_sess.trial_matrices['spks']
    trialmat[np.isnan(trialmat)]=1E-3
    
    cellmask = fam_place_cell_mask(concat_sess)+nov_place_cell_mask(concat_sess)>0
    
    column = 0
    plot_arrays = np.zeros([2,18,trialmat.shape[1],cellmask.sum()])
    for day in np.unique(concat_sess.trial_info['sess_num']).tolist():
        for block in range(6):
            mask = (concat_sess.trial_info['sess_num']==day)*(concat_sess.trial_info['block_number']==block)
            l_mask = mask*(concat_sess.trial_info['LR']==-1)
            l_tmp = trialmat[l_mask,:,:].mean(axis=0)
            l_tmp = sp.stats.zscore(l_tmp[:, cellmask],axis=0)

            r_mask = mask*(concat_sess.trial_info['LR']==1)
            r_tmp = trialmat[r_mask,:,:].mean(axis=0)
            r_tmp = sp.stats.zscore(r_tmp[:, cellmask],axis=0)
            
            plot_arrays[0,column,:,:] = l_tmp
            plot_arrays[1,column,:,:] = r_tmp
            
            column +=1
    return plot_arrays

def rev_pc_plot(plot_arrays):
    fig, ax = plt.subplots(2, 18, figsize = [36,6])
    for col in range(18):
            l_tmp, r_tmp = [], []
            for mouse, plot_arr in plot_arrays.items():
                l_tmp.append(plot_arr[0, col, :, :])
                r_tmp.append(plot_arr[1, col, :, :])
            l_tmp = np.concatenate(l_tmp,axis=-1)
            r_tmp = np.concatenate(r_tmp,axis=-1)

            if col == 0:
                l_sortvec = np.argsort(np.argmax(l_tmp, axis=0))
                r_sortvec = np.argsort(np.argmax(r_tmp, axis=0))

            ax[0, col].imshow(l_tmp[:,l_sortvec].T,cmap='pink',aspect='auto',vmin=0, vmax=2)
            ax[1, col].imshow(r_tmp[:,r_sortvec].T,cmap='pink',aspect='auto',vmin=0, vmax=2)
            



ko_plot_arrays = { mouse: get_rev_plot_arr(u.single_mouse_concat_sessions(mouse,date_inds=[6, 7, 8])) for mouse in stx.ymaze_sess_deets.KO_sessions.keys() if mouse not in ['4467975.1']}
ctrl_plot_arrays = { mouse: get_rev_plot_arr(u.single_mouse_concat_sessions(mouse,date_inds=[6, 7, 8])) for mouse in stx.ymaze_sess_deets.CTRL_sessions.keys()}


In [ ]:
rev_pc_plot(ko_plot_arrays)
rev_pc_plot(ctrl_plot_arrays)

In [ ]:
def rev_pc_plot_endpoints(plot_arrays):
    fig, ax = plt.subplots(2, 4, figsize = [12,4])
    fig.subplots_adjust(wspace=.5)
    for j, col in enumerate([0,1, 11, 11]):
            l_tmp, r_tmp = [], []
            for mouse, plot_arr in plot_arrays.items():
                l_tmp.append(plot_arr[0, col, :, :])
                r_tmp.append(plot_arr[1, col, :, :])
            l_tmp = np.concatenate(l_tmp,axis=-1)
            r_tmp = np.concatenate(r_tmp,axis=-1)

            if col == 0:
                l_sortvec = np.argsort(np.argmax(l_tmp, axis=0))
                r_sortvec = np.argsort(np.argmax(r_tmp, axis=0))
            if j == 3:
                l_sortvec = np.argsort(np.argmax(l_tmp, axis=0))
                r_sortvec = np.argsort(np.argmax(r_tmp, axis=0))
                
            ax[0, j].imshow(l_tmp[:,l_sortvec].T,cmap='pink',aspect='auto',vmin=0, vmax=2)
            ax[1, j].imshow(r_tmp[:,r_sortvec].T,cmap='pink',aspect='auto',vmin=0, vmax=2)
    return fig

ko_fig = rev_pc_plot_endpoints(ko_plot_arrays)
ctrl_fig = rev_pc_plot_endpoints(ctrl_plot_arrays)




ko_fig.savefig(os.path.join(figdir,"ko_reversal_endpoints.pdf"))
ctrl_fig.savefig(os.path.join(figdir,"ctrl_reversal_endpoints.pdf"))

In [ ]:
def corr_to_baseline_rev(concat_sess):
    trialmat = concat_sess.trial_matrices['spks']
    trialmat[np.isnan(trialmat)] = 1E-3
    trialmat = sp.ndimage.filters.gaussian_filter1d(trialmat[:,:,:],1,axis=1)

    baseline_mask = (concat_sess.trial_info['sess_num']==6)*(concat_sess.trial_info['block_number']<2)*(concat_sess.trial_info['LR']==-1)
    baseline = trialmat[baseline_mask,:,:].mean(axis=0)
    baseline = sp.stats.zscore(baseline,axis=-1)[np.newaxis,:,:]

    test_mask = concat_sess.trial_info['LR']==-1
    test_mask[(concat_sess.trial_info['sess_num']==6)*(concat_sess.trial_info['block_number']<2)] = False
    test_mask[(concat_sess.trial_info['sess_num']==8)*(concat_sess.trial_info['block_number']>2)] = False
    test = trialmat[test_mask,:,:]
    test = sp.stats.zscore(test,axis=-1)


    _corr_to_baseline = 1./test.shape[-1]*np.matmul(baseline,np.transpose(test,axes=(0,2,1)))
    corr_to_baseline = np.array([np.diag(_corr_to_baseline[trial,:,:]).mean() for trial in range(test.shape[0])])
    return corr_to_baseline

def corr_to_endpoint_rev(concat_sess):
    trialmat = concat_sess.trial_matrices['spks']
    trialmat[np.isnan(trialmat)] = 1E-3
    trialmat = sp.ndimage.filters.gaussian_filter1d(trialmat[:,:,:],1,axis=1)

    endpoint_mask = (concat_sess.trial_info['sess_num']==8)*(concat_sess.trial_info['block_number']<2)*(concat_sess.trial_info['LR']==-1)
    endpoint = trialmat[endpoint_mask,:,:].mean(axis=0)
    endpoint = sp.stats.zscore(endpoint,axis=-1)[np.newaxis,:,:]

    test_mask = (concat_sess.trial_info['LR']==-1) *(concat_sess.trial_info['sess_num']==6)*(concat_sess.trial_info['block_number']>2)
    test_mask[(concat_sess.trial_info['sess_num']==8)*(concat_sess.trial_info['block_number']>-1)] = False
    test = trialmat[test_mask,:,:]
    test = sp.stats.zscore(test,axis=-1)


    _corr_to_endpoint = 1./test.shape[-1]*np.matmul(endpoint,np.transpose(test,axes=(0,2,1)))
    corr_to_endpoint = np.array([np.diag(_corr_to_endpoint[trial,:,:]).mean() for trial in range(test.shape[0])])
    return corr_to_endpoint

def corr_ratio_rev(concat_sess, metric = 'corr', cellavg = True):
    if not cellavg:
        trialmat = concat_sess.trial_matrices['spks_norm']
    else:
        trialmat = concat_sess.trial_matrices['spks']
    trialmat[np.isnan(trialmat)] = 1E-3
    trialmat = sp.ndimage.filters.gaussian_filter1d(trialmat[:,:,:],1,axis=1)
    lickmat = concat_sess.trial_matrices['licks']
#     lickmat[np.isnan(lickmat)]=0.

    res = {}
    for lr in [-1,1]:
    
        baseline_mask = (concat_sess.trial_info['sess_num']==6)*(concat_sess.trial_info['block_number']<2)*(concat_sess.trial_info['LR']==lr)
        baseline_mask[concat_sess.trial_info['sess_num']<6]=True
        baseline = trialmat[baseline_mask,:,:].mean(axis=0,keepdims=True)
        if lr == -1:
            print('lr', concat_sess.rzone_early['t_antic'], concat_sess.rzone_early['tfront'])
            rzone_mask = (concat_sess.trial_matrices['bin_edges'][:-1]>=concat_sess.rzone_early['t_antic']) * (concat_sess.trial_matrices['bin_edges'][1:]<=concat_sess.rzone_early['tfront'])
        elif lr == 1:
            rzone_mask = (concat_sess.trial_matrices['bin_edges'][:-1]>=concat_sess.rzone_late['t_antic']) * (concat_sess.trial_matrices['bin_edges'][1:]<=concat_sess.rzone_late['tfront'])
        
        

        endpoint_mask = (concat_sess.trial_info['sess_num']==8)*(concat_sess.trial_info['block_number']<2)*(concat_sess.trial_info['LR']==lr)
        endpoint_mask[(concat_sess.trial_info['sess_num']==7)*(concat_sess.trial_info['block_number']==5)*(concat_sess.trial_info['LR']==lr)] = True
        endpoint = trialmat[endpoint_mask,:,:].mean(axis=0,keepdims=True)


        test_mask = (concat_sess.trial_info['LR']==lr) *(concat_sess.trial_info['sess_num']==6)*(concat_sess.trial_info['block_number']>2)
        test_mask[(concat_sess.trial_info['sess_num']==7)*(concat_sess.trial_info['block_number']<5)*(concat_sess.trial_info['LR']==lr)] = True
        test = trialmat[test_mask,:,:]
        
        
        
        
        
        assert (metric in ('corr', 'cos')), "wrong metric"
                
        if metric == 'corr':
            if cellavg:
                baseline = sp.stats.zscore(baseline,axis=1)
                endpoint = sp.stats.zscore(endpoint,axis=1)
                test = sp.stats.zscore(test,axis=1)


                _corr_to_baseline = 1./test.shape[1]*np.matmul(np.transpose(test,axes=(0,2,1)),baseline)   
                _corr_to_endpoint = 1./test.shape[1]*np.matmul(np.transpose(test,axes=(0,2,1)),endpoint)
                
                corr_base_end = np.diag(1./test.shape[1]*np.squeeze(np.matmul(np.transpose(baseline,axes=(0,2,1)),endpoint)))
            else:
                baseline = sp.stats.zscore(baseline,axis=-1)
                endpoint = sp.stats.zscore(endpoint,axis=-1)
                test = sp.stats.zscore(test,axis=-1)


                _corr_to_baseline = 1./test.shape[1]*np.matmul(baseline, np.transpose(test,axes=(0,2,1)))   
                _corr_to_endpoint = 1./test.shape[1]*np.matmul(endpoint, np.transpose(test,axes=(0,2,1)))
                
                corr_base_end = np.diag(1./test.shape[1]*np.squeeze(np.matmul(endpoint, np.transpose(baseline,axes=(0,2,1)))))
 
        elif metric == 'cos':
            if cellavg:
                baseline /= np.linalg.norm(baseline,ord=2,axis=1, keepdims=True) + 1E-3
                endpoint /= np.linalg.norm(endpoint,ord=2,axis=1, keepdims=True) + 1E-3
                test /= np.linalg.norm(test,ord=2,axis=1, keepdims=True) + 1E-3
            
                _corr_to_baseline = np.matmul(np.transpose(test,axes=(0,2,1)),baseline)
                _corr_to_endpoint = np.matmul(np.transpose(test,axes=(0,2,1)),endpoint)
                
                corr_base_end = np.diag(np.squeeze(np.matmul(np.transpose(baseline,axes=(0,2,1)),endpoint)))
            else:
                baseline /= np.linalg.norm(baseline,ord=2,axis=-1, keepdims=True) + 1E-3
                endpoint /= np.linalg.norm(endpoint,ord=2,axis=-1, keepdims=True) + 1E-3
                test /= np.linalg.norm(test,ord=2,axis=-1, keepdims=True) + 1E-3
            
                _corr_to_baseline = np.matmul(baseline, np.transpose(test,axes=(0,2,1)))
                _corr_to_endpoint = np.matmul(endpoint, np.transpose(test,axes=(0,2,1)))
   
                corr_base_end = np.diag(np.squeeze(np.matmul(endpoint, np.transpose(baseline,axes=(0,2,1)))))
                
            
            
            
        corr_to_baseline = np.array([np.diag(_corr_to_baseline[trial,:,:]).mean() for trial in range(test.shape[0])])
        corr_to_endpoint = np.array([np.diag(_corr_to_endpoint[trial,:,:]).mean() for trial in range(test.shape[0])])
        
        res[lr]= {'trial': -1*np.log(np.maximum(1E-3,corr_to_endpoint)) + np.log(np.maximum(1E-3,corr_to_baseline)), 
                  'endpoints': corr_base_end}
        
    return res




ko_corr_ratio = {mouse:corr_ratio_rev(single_mouse_concat_sessions(mouse,date_inds=[6, 7]),metric='cos', cellavg=True) for mouse in stx.ymaze_sess_deets.KO_sessions.keys()} # if mouse not in ['4467975.1']}
ctrl_corr_ratio = {mouse:corr_ratio_rev(single_mouse_concat_sessions(mouse,date_inds=[6, 7]),metric='cos', cellavg=True) for mouse in stx.ymaze_sess_deets.CTRL_sessions.keys()}
